# DATAIKU interview

* Provided Data : US Census dataset
* Goal : Prediction task is to determine whether a person makes over 50K a year. 

# Import  Python Libraries and packages

In [1]:
import csv, pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, math
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model, metrics, model_selection, neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score 
import time

# Import data 
Train, test and attributs 

In [2]:
attributs = ['age','class of worker','detailed industry recode',                                                                                           
'detailed occupation recode','education','wage per hour',                                                                                                      
'enroll in edu inst last wk','marital stat','major industry code',                                                                                                
'major occupation code','race','hispanic origin', 'sex',                                                                                                                
'member of a labor union','reason for unemployment',
'full or part time employment stat','capital gains',
'capital losses','dividends from stocks',
'tax filer stat','region of previous residence',
'state of previous residence',
'detailed household and family stat',
'detailed household summary in household',
'migration code-change in msa','migration code-change in reg',
'migration code-move within reg','live in this house 1 year ago',
'migration prev res in sunbelt','num persons worked for employer',
'family members under 18','presence of family members', 
'country of birth father','country of birth mother',
'country of birth self','citizenship','own business or self employed',
'fill inc questionnaire for veterans admin','veterans benefits',
'weeks worked in year','year','GT']

print("Number of attributs : {}".format(len(attributs)))

data_path = '/home/mirosky/dev/dataiku_interview/final/data/'
train_filename = data_path + "census_income_learn.csv"
test_filename = data_path + "census_income_test.csv"

train_data  = pd.read_csv(train_filename, header=None, names=attributs, engine='python')
test_data  = pd.read_csv(test_filename, header=None, names=attributs, engine='python')

# Quick overview on the provided data.
print("Size of training data set : {} and testing data set : {}".format(train_data.shape, test_data.shape))
with pd.option_context('display.max_rows', 3, 'display.max_columns', train_data.shape[1]):
     display(train_data.head())

Number of attributs : 42
Size of training data set : (199523, 42) and testing data set : (61342, 42)


,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,presence of family members,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veterans admin,veterans benefits,weeks worked in year,year,GT
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


# Pruning
Elimination of some attributs and associated data (see report for explanations)

In [3]:
attr_to_remove = ['detailed industry recode',                                                                                           
'detailed occupation recode', 'enroll in edu inst last wk','major industry code',                                                                                                
'hispanic origin', 'member of a labor union','reason for unemployment',
'full or part time employment stat','tax filer stat','region of previous residence',
'detailed household and family stat','detailed household summary in household',
'migration code-change in reg','migration code-move within reg','live in this house 1 year ago',
'migration prev res in sunbelt','family members under 18','presence of family members', 
'country of birth father','country of birth mother','country of birth self',
'fill inc questionnaire for veterans admin','veterans benefits','num persons worked for employer']

train_data_filtered = train_data.drop(attr_to_remove, axis=1, inplace=False)
attr_filtered = attributs.copy()

for i in attr_to_remove:
    if i in attr_filtered :
        attr_filtered.remove(i)
        
print("Filtered attributs ( {} ) : \n {} \n Size of filtered training data set : {} "
      .format(len(attr_filtered), np.transpose(attr_filtered), train_data_filtered.shape))

Filtered attributs ( 18 ) : 
 ['age' 'class of worker' 'education' 'wage per hour' 'marital stat'
 'major occupation code' 'race' 'sex' 'capital gains' 'capital losses'
 'dividends from stocks' 'state of previous residence'
 'migration code-change in msa' 'citizenship'
 'own business or self employed' 'weeks worked in year' 'year' 'GT'] 
 Size of filtered training data set : (199523, 18) 


# Exploration
Study the quality of the data, in particular the missing data : "Not in universe"

In [4]:
# Study the columns with missing values and delete those that contains too much.

# Load all data
train_file_all = pd.read_csv(train_filename, header = None, skipinitialspace = True)
train_data_all = pd.DataFrame(train_file_all)
 
# Load only data without missing values
# NaN values are put if "Not in universe" in each column.
train_data_without_miss = pd.read_csv(train_filename, header = None,
                          skipinitialspace = True, na_values= "Not in universe")
train_data_without_miss = pd.DataFrame(train_data_without_miss)

pourc_good_data = (train_data_without_miss.count()/(train_data_all.count())*100.)
print("Pourcentage for each attributs of ""no missing"" data : {} \n ".format((pourc_good_data.values)))

# Define a threshold of 40% and delete a column if there is less than 40% of good values.
index_toomuch_miss = pourc_good_data[pourc_good_data < 40]
train_data_filtered_2 = train_data_filtered.copy()
attr_to_remove_2 = []

for i in index_toomuch_miss.index : 
    print("Attribut : {} with purcentage : {} must be removed. ".format(attributs[i], pourc_good_data[i]))
    attr_to_remove_2.append(attributs[i])
    if attributs[i] in train_data_filtered_2 : 
        print("Removed.")
        train_data_filtered_2 = train_data_filtered_2.drop(attributs[i], axis=1)
    else:
        print("Already removed.")
        
attr_filtered_2 = attr_filtered.copy()

for i in attr_to_remove_2:
    if i in attr_filtered_2 :
        print("\n Remove attribut : {}".format(i))
        attr_filtered_2.remove(i)
        

Pourcentage for each attributs of no missing data : [ 100.           49.75767205  100.          100.          100.          100.
    6.30503751  100.          100.           49.53764729  100.          100.
  100.            9.55478817    3.04225578  100.          100.          100.
  100.          100.            7.90535427    7.90535427  100.          100.
  100.           99.24018785   99.24018785   99.24018785  100.
   57.87252597  100.           27.71159215  100.          100.          100.
  100.          100.            0.99437158  100.          100.          100.
  100.        ] 
 
Attribut : enroll in edu inst last wk with purcentage : 6.305037514472016 must be removed. 
Already removed.
Attribut : member of a labor union with purcentage : 9.554788169784938 must be removed. 
Already removed.
Attribut : reason for unemployment with purcentage : 3.0422557800353847 must be removed. 
Already removed.
Attribut : region of previous residence with purcentage : 7.905354269933793 must b

In [5]:
# re affectation of variables and display filtered data.
attributs = attr_filtered_2.copy()
train_data = train_data_filtered_2.copy()

print("Filtered attributs ( {} ) : \n\n {} \n\n Size of filtered training data set : {} "
      .format(len(attributs), np.transpose(attributs), train_data.shape))

Filtered attributs ( 17 ) : 

 ['age' 'class of worker' 'education' 'wage per hour' 'marital stat'
 'major occupation code' 'race' 'sex' 'capital gains' 'capital losses'
 'dividends from stocks' 'migration code-change in msa' 'citizenship'
 'own business or self employed' 'weeks worked in year' 'year' 'GT'] 

 Size of filtered training data set : (199523, 17) 


# Statistical audit
* Based on the learning file:
* Make a quick statistic based and univariate audit of the different columns’ content and produce the results in visual / graphic format.
* This audit should describe the variable distribution, the % of missing values, the extreme values, and so on.

In [6]:
# Continuous variables 
train_data.describe()

,age,wage per hour,capital gains,capital losses,dividends from stocks,migration code-change in msa,own business or self employed,weeks worked in year,year
count,199523.000000,199523.000000,199523.00000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000
mean,34.494199,55.426908,434.71899,37.313788,197.529533,1740.380269,0.175438,23.174897,94.499672
std,22.310895,274.896454,4697.53128,271.896428,1984.163658,993.768156,0.553694,24.411488,0.500001
min,0.000000,0.000000,0.00000,0.000000,0.000000,37.870000,0.000000,0.000000,94.000000
25%,15.000000,0.000000,0.00000,0.000000,0.000000,1061.615000,0.000000,0.000000,94.000000
50%,33.000000,0.000000,0.00000,0.000000,0.000000,1618.310000,0.000000,8.000000,94.000000
75%,50.000000,0.000000,0.00000,0.000000,0.000000,2188.610000,0.000000,52.000000,95.000000
max,90.000000,9999.000000,99999.00000,4608.000000,99999.000000,18656.300000,2.000000,52.000000,95.000000


In [7]:
# Categorical variables
train_data.describe(include=['O'])

,class of worker,education,marital stat,major occupation code,race,sex,citizenship,GT
count,199523,199523,199523,199523,199523,199523,199523,199523
unique,9,17,7,15,5,2,5,2
top,Not in universe,High school graduate,Never married,Not in universe,White,Female,Native- Born in the United States,- 50000.
freq,100245,48407,86485,100684,167365,103984,176992,187141


In [8]:
# Continuous variables per class (> or <= 50K)
display(train_data.groupby(['GT']).mean())
display(train_data.groupby(['GT']).std())

,age,wage per hour,capital gains,capital losses,dividends from stocks,migration code-change in msa,own business or self employed,weeks worked in year,year
GT,,,,,,,,,
- 50000.,33.715316,53.692526,143.848013,27.003730,107.816518,1736.683288,0.169674,21.527762,94.497769
50000+.,46.266193,81.640284,4830.930060,193.139557,1553.448070,1796.256279,0.262559,48.069617,94.528428


,age,wage per hour,capital gains,capital losses,dividends from stocks,migration code-change in msa,own business or self employed,weeks worked in year,year
GT,,,,,,,,,
- 50000.,22.620236,261.168380,1815.343341,229.535067,909.774217,995.694075,0.547173,24.118209,0.499996
50000+.,11.830906,431.364773,16887.627002,607.542507,6998.071762,962.502889,0.637988,12.259412,0.499211


In [9]:
# Delete attributs that have almost same values in both classes in mean and no huge std.

attr_to_remove_3 = ['migration code-change in msa','year', 'citizenship']
train_data_filtered_3 = train_data.drop(attr_to_remove_3, axis=1)
attr_filtered_3 = attributs.copy()

for i in attr_to_remove_3:
    if i in attr_filtered_3 :
        print("\n Remove attribut : {}".format(i))
        attr_filtered_3.remove(i) 


 Remove attribut : migration code-change in msa

 Remove attribut : year

 Remove attribut : citizenship


In [10]:
# re affectation of variables and display.
attributs = attr_filtered_3.copy()
train_data = train_data_filtered_3.copy()

print("Filtered attributs ( {} ) : \n\n {} \n\n Size of filtered training data set : {} "
      .format(len(attributs), np.transpose(attributs), train_data.shape))


Filtered attributs ( 14 ) : 

 ['age' 'class of worker' 'education' 'wage per hour' 'marital stat'
 'major occupation code' 'race' 'sex' 'capital gains' 'capital losses'
 'dividends from stocks' 'own business or self employed'
 'weeks worked in year' 'GT'] 

 Size of filtered training data set : (199523, 14) 


In [11]:
# Plot distributions of variables in an independant way
 
nb_col=5  
fig = plt.figure(figsize=(50,30))
fig.subplots_adjust(wspace=0.8, hspace=0.4)
rows = math.ceil(float(train_data.shape[1]) / nb_col)

for i, attr in enumerate(train_data.columns):
    ax = fig.add_subplot(rows, nb_col, i + 1)
    ax.set_title(attr)
    if train_data.dtypes[attr] == np.object:
        g = sns.countplot(y=attr, data=train_data)
        substrings = [s.get_text()[:40] for s in g.get_yticklabels()]
        g.set(yticklabels=substrings)
    else:
        g = sns.distplot(train_data[attr])

plt.savefig('images/distributions.png')
plt.close()

Study variables function of each class.


In [12]:
# Plot distributions of variables function of each class.

# Make some bins for somes variables.
train_data_final = train_data.copy() 
train_data_final['age'] = pd.cut(train_data_final['age'], 10) 
train_data_final['capital gains'] = pd.cut(train_data_final['capital gains'], 10) 
train_data_final['capital losses'] = pd.cut(train_data_final['capital losses'], 10) 

nb_col=5 
fig = plt.figure(figsize=(50,30))
fig.subplots_adjust(wspace=0.8, hspace=0.4)
rows = math.ceil(float(train_data_final.shape[1]) / nb_col)

for i, attr in enumerate(train_data_final.columns):
    ax = fig.add_subplot(rows, nb_col, i + 1)
    ax.set_title(attr)
#     if train_data.dtypes[attr] == np.object:
    g = sns.countplot(y=attr, hue='GT', data=train_data_final)
    substrings = [s.get_text()[:40] for s in g.get_yticklabels()]
    g.set(yticklabels=substrings)

plt.savefig('images/distributions_per_class.png')
plt.close()

Study correlation between variables.

In [13]:
# Before plotting, data must be all continues.
 
train_data_final.loc[train_data['GT'] == ' - 50000.', 'GT'] = 0 # negative class
train_data_final.loc[train_data['GT'] == ' 50000+.', 'GT'] = 1 # positive class

train_encoded = train_data_final.apply(LabelEncoder().fit_transform)
print("Training data encoded with only continous values size : {}".format(train_encoded.shape))

Training data encoded with only continous values size : (199523, 14)


In [14]:
# Plot correlation matrix.

fig = plt.figure(figsize=(50,30)) 
matrix = np.zeros_like(train_encoded.corr(), dtype=np.bool)
matrix[np.triu_indices_from(matrix)] = True
sns.heatmap(train_encoded.corr(), vmin=-1, vmax=1, square=True, mask=matrix,
            linewidths=.9, cmap= sns.color_palette("RdGy",200));
 
plt.savefig('./images/correlation_matrix.png.')
plt.close()

# Step 3 : Model building
* Create a model using these variables to modelize wining more or less than $50,000 / year.
* Here, the idea would be for you to test one or two algorithms, type regression logistics, or a decision tree. But, you are free to choose others if you’d rather.

Keep a part of the training dataset to evaluate the algorithm

In [15]:
# Splitting the Training and Test data sets
size_for_test = train_encoded.shape[0]/3

val = train_encoded.loc[0:size_for_test,:]
train = train_encoded.loc[size_for_test:,:]

print("Training dataset size : {}".format(train.shape))
print("Validation dataset size : {}".format(val.shape))

# Check NaN values
train = train.dropna(axis=0)
val = val.dropna(axis=0)

# Fill parameters for logistical regression
X_train = train.drop(['GT'], axis=1)
Y_train = train['GT'].astype('int64')

X_val  = val.drop(['GT'], axis=1)
Y_val  = val['GT'].astype('int64')

Training dataset size : (133015, 14)
Validation dataset size : (66508, 14)


Training ("fitting") part and score on validation dataset with different classification algorithms  

# Logistic regression


In [16]:
# score on the validation dataset
start_time=time.time()

logistic = linear_model.LogisticRegression() 
model = logistic.fit(X_train, Y_train)
print("Parameters logistic regression model : {} ".format(model))
Y_predict_val = model.predict(X_val)
print(metrics.classification_report(Y_val, Y_predict_val))

print("Time : {}".format(time.time()-start_time ))

# cross validation scores with K = 10 folds
start_time=time.time()

predict_train = model_selection.cross_val_predict(logistic,  X_train, Y_train, cv=10, n_jobs = -1)
print(metrics.classification_report(Y_train, predict_train))

print("Time : {}".format(time.time()-start_time ))

Parameters logistic regression model : LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) 
             precision    recall  f1-score   support

          0       0.95      0.99      0.97     62427
          1       0.71      0.22      0.34      4081

avg / total       0.94      0.95      0.93     66508

Time : 2.140747547149658
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    124714
          1       0.72      0.23      0.34      8301

avg / total       0.94      0.95      0.93    133015

Time : 13.82686734199524


# KNN


In [ ]:
# score on the validation dataset
start_time=time.time()

knn = neighbors.KNeighborsClassifier()
model = knn.fit(X_train, Y_train)
print("Parameters KNN model : {} ".format(model))
Y_predict_val = model.predict(X_val)
print(metrics.classification_report(Y_val, Y_predict_val))

print("Time : {}".format(time.time()-start_time ))

# cross validation scores with K = 10 folds
start_time=time.time()

predict_train = model_selection.cross_val_predict(logistic,  X_train, Y_train, cv=10, n_jobs = -1)
print(metrics.classification_report(Y_train, predict_train))

print("Time : {}".format(time.time()-start_time ))

Parameters KNN model : KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform') 


# Application of the model

Choose the model that appears to have the highest performance based on a comparison between reality (the 42nd variable) and the model’s prediction.

In [26]:
# Re-train on the whole dataset

train_all = train_encoded.copy()
train_all = train_all.dropna(axis=0)
print("Training dataset all size : {}".format(train_all.shape))

X_train_all = train_all.drop(['GT'], axis=1)
Y_train_all = train_all['GT'].astype('int64')

start_time=time.time()

logistic = linear_model.LogisticRegression() 
model = logistic.fit(X_train_all, Y_train_all)
print("Parameters logistic regression model : {} ".format(model))

# Build features for test data i.e. delete attributs in the test dataset that was not used for training.

attr_to_remove_all = attr_to_remove + attr_to_remove_2 + attr_to_remove_3
test_data_all = pd.DataFrame(test_data)
test_data_filtered = test_data_all.drop(attr_to_remove_all, axis=1)
test_data_filtered = test_data_filtered.dropna(axis=0)

test_enc = test_data_filtered.apply(LabelEncoder().fit_transform)
X_real_test  = test_enc.drop(['GT'], axis=1)
Y_real_test  = test_enc['GT'].astype('int64')

# Apply trained model on the real test dataset.
Y_predict_real = model.predict(X_real_test)
print(metrics.classification_report(Y_real_test, Y_predict_real))


Training dataset size : (199523, 14)
Parameters logistic regression model : LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) 
             precision    recall  f1-score   support

          0       1.00      0.34      0.51     57525
          1       0.09      1.00      0.17      3816

avg / total       0.94      0.38      0.49     61341

